In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\222690\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\222690\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [ ]:
df = pd.read_csv("./data/recipes_sample.csv")
five_values = df.sample(n = 5)[['id','minutes']]
five_values.reset_index(inplace=True)
max_id = len(str(max(df['id']))) + 8
max_min = max(len(str(max(df['minutes']))), len('minutes')) + 4
print(f"|{'id':^{max_id}}|{'minutes':^{max_min}}|")
print('|'+'-'*(max_id+max_min+1)+'|')
for i in range(5):
    print(f'|{five_values["id"][i]:^{max_id}}|{five_values["minutes"][i]:^{max_min}}|')

|      id      |  minutes  |
|--------------------------|
|    239961    |    35     |
|    146271    |    75     |
|    200484    |    30     |
|    154823    |    20     |
|    338200    |    60     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__).
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
with open('./data/steps_sample.xml', 'r') as file:
    recipes = BeautifulSoup(file.read(), 'xml')
recipe_text = recipes.find('id', string='170895').next_sibling.next_sibling.text.strip().split('\n')

r_pd = pd.read_csv('./data/recipes_sample.csv')
print(r_pd.head())
looked_data = r_pd[r_pd['id'] == 170895]
print('-'*150)
print(looked_data.head())
name_recipe, minutes, author_id = looked_data['name'].values[0], looked_data['minutes'].values[0], looked_data['contributor_id'].values[0]


def show_info(name:str,
              steps: list[str],
              minutes:int,
              author_id:int) -> str:
    res_str = f'"{name.title()}"\n\n'
    res_str += "\n".join([f"{i+1}. {v.capitalize()}" for i, v in enumerate(steps)])
    res_str += "\n"+"-"*10+"\n"
    res_str += f"Автор: {author_id}\n"
    res_str += f"Среднее время приготовления: {minutes} минут\n"
    return res_str

print(show_info(name_recipe, recipe_text, minutes, author_id))

                                       name     id  minutes  contributor_id  \
0     george s at the cove  black bean soup  44123       90           35193   
1        healthy for them  yogurt popsicles  67664       10           91970   
2              i can t believe it s spinach  38798       30            1533   
3                      italian  gut busters  35173       45           22724   
4  love is in the air  beef fondue   sauces  84797       25            4470   

    submitted  n_steps                                        description  \
0  2002-10-25      NaN  an original recipe created by chef scott meska...   
1  2003-07-26      NaN  my children and their friends ask for my homem...   
2  2002-08-29      NaN            these were so go, it surprised even me.   
3  2002-07-27      NaN  my sister-in-law made these for us at a family...   
4  2004-02-23      4.0  i think a fondue is a very romantic casual din...   

   n_ingredients  
0           18.0  
1            NaN  
2    

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
pattern = r"(\d+\s(hour[s]*|minute[s]*))"
data_recipes = recipes.find('id', string='25082').next_sibling.next_sibling.text.strip().split('\n')
res = re.findall(pattern, '\n'.join(data_recipes))
for s in res:
    print(s[0])

20 minutes
10 minutes
2 hours
10 minutes
20 minutes
30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
pattern = r'^this.*,[\s]?but'
descriptions = pd.Series(df['description']).dropna()
descriptions_pattern = descriptions.str.match(pattern)
print(f"Кол-во рецептов: {descriptions_pattern.sum()}")
descriptions[descriptions_pattern][:3]

Кол-во рецептов: 865


76    this is a great meal eaten the same day ,but e...
87    this is it. the real deal. very time consuming...
92    this is fantastic served with grilled seafood ...
Name: description, dtype: object

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
data_recipes = recipes.find('id', string='72367').next_sibling.next_sibling.text.strip()
print(re.sub(r' / ', '/', data_recipes))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
unique_words = set()
for word in recipes.find_all('recipe'):
    unique_words.update(nltk.word_tokenize(word.steps.text.strip()))
len(unique_words)

19604

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
df['description'] = df['description'].dropna()
descriptions = df["description"].apply(nltk.sent_tokenize)
data = descriptions.to_dict()
df.iloc[sorted(data, key=lambda x: len(data[x]), reverse=True)[:5]]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
```
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
pos_tag_values = nltk.pos_tag(nltk.word_tokenize(df[df['id'] == 241106].name.values[0]))
formatted_values = [(value[-1], value[-2], max(len(value[0]), len(value[1]))) for value in pos_tag_values]
print(''.join([f"{ind[0]:^{ind[2] + 1}}" for ind in formatted_values]),
      ''.join([f"{ind[1]:^{ind[2] + 1}}" for ind in formatted_values]),
      sep='\n')

   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
